In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [3]:
!pip install peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 112.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 117.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [4]:
# Install and import the necessary libraries
# !pip install torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops 

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [5]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-finetuned"

In [6]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch

def load_and_sample_snli():
    dataset = load_dataset("stanfordnlp/snli")
    
    # Sampling every 550th sample for training, every 100th for test and validation
    train_samples = dataset['train'].select(range(0, len(dataset['train']), 550))[:1000]
    test_samples = dataset['test'].select(range(0, len(dataset['test']), 100))[:100]
    validation_samples = dataset['validation'].select(range(0, len(dataset['validation']), 100))[:100]
    
    return DatasetDict({
        'train': Dataset.from_dict(train_samples),
        'test': Dataset.from_dict(test_samples),
        'validation': Dataset.from_dict(validation_samples)
    })

dataset = load_and_sample_snli()


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [8]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
def generate_prompt(premise, hypothesis):
    return (
        f"Read the following Premise and Hypothesis and determine whether Hypothesis entails the Premise or not.\n"
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        f"Choose the best answer:\n"
        f"If Hypothesis entails the Premise, Answer: 0\n"
        f"If Neutral, Answer: 1\n"
        f"If Hypothesis does not entail the Premise, Answer: 2\n"
        f"Answer only one numerical value. \n\n"
        f"Answer:"
    )

In [10]:
import re

# Define mapping of labels
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
pretrain_outputs = []
# Evaluate on the test set
correct = 0
total = 0

for example in dataset['test']:
    # Create prompt from premise and hypothesis
    prompt = generate_prompt(example['premise'], example['hypothesis'])
    
    # Generate model's response
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=15)
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip().lower()
#     print(f"Model response: {response}")
    
    # Find the last occurrence of "answer: 0", "answer: 1", or "answer: 2" in reverse order
    last_answer = None

    for ans in ["answer: 0", "answer: 1", "answer: 2"]:
        pos = response.rfind(ans)  # Search from the end of the text
        if pos != -1:
            # Keep track of the last found answer if it appears later in the text
            if last_answer is None or pos > response.rfind(last_answer):
                last_answer = ans

    output = None
    if(last_answer == "answer: 0"):
        output = 0
    elif(last_answer == "answer: 1"):
        output = 1
    elif(last_answer == "answer: 2"):
        output = 2
    
    if output == example['label']:
        correct += 1
    total += 1
    pretrain_outputs.append(output)
# Calculate and print accuracy
accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

Accuracy: 0.5000


In [11]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)


In [12]:
# LoRA configuration
peft_config = LoraConfig(
    r= 32,          
    lora_alpha= 32,
    lora_dropout=0.07,
    bias="none",
    task_type="CAUSAL_LM",
)

In [13]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    gradient_checkpointing=True,
    gradient_accumulation_steps=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",       # Log at specific steps 
    learning_rate=1e-4,
    group_by_length = True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    warmup_steps = 5,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch"
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Define the function to generate prompt
def generate_prompt(example):
#     print(example)
    premise, hypothesis, label = example['premise'], example['hypothesis'], example['label']
    prompt = (
        f"Read the following Premise and Hypothesis and determine whether Hypothesis entails the Premise or not.\n"
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        f"Choose the best answer:\n"
        f"If Hypothesis entails the Premise, Answer: 0\n"
        f"If Neutral, Answer: 1\n"
        f"If Hypothesis does not entail the Premise, Answer: 2\n"
        f"Answer only one numerical value. \n\n"
        f"Answer: {label if label is not None else ''}"
    )
    return prompt

train_data = {"prompt": [generate_prompt(example) for example in dataset['train']], "label": [example["label"] for example in dataset['train']]}
val_data = {"prompt": [generate_prompt(example) for example in dataset['validation']], "label": [example["label"] for example in dataset['validation']]}
test_data = {"prompt": [generate_prompt(example) for example in dataset['test']], "label": [example["label"] for example in dataset['test']]}

dataset_prompt = DatasetDict({
    "train": Dataset.from_dict(train_data),
    "val": Dataset.from_dict(val_data),
    "test": Dataset.from_dict(test_data)
})

# Let's print a sample from each to verify the format
print("Sample from Train Dataset:\n", dataset_prompt['train'][0])
print("\nSample from Validation Dataset:\n", dataset_prompt['val'][0])
print("\nSample from Test Dataset:\n", dataset_prompt['test'][0])


Sample from Train Dataset:
 {'prompt': 'Read the following Premise and Hypothesis and determine whether Hypothesis entails the Premise or not.\nPremise: A person on a horse jumps over a broken down airplane.\nHypothesis: A person is training his horse for a competition.\nChoose the best answer:\nIf Hypothesis entails the Premise, Answer: 0\nIf Neutral, Answer: 1\nIf Hypothesis does not entail the Premise, Answer: 2\nAnswer only one numerical value. \n\nAnswer: 1', 'label': 1}

Sample from Validation Dataset:
 {'prompt': 'Read the following Premise and Hypothesis and determine whether Hypothesis entails the Premise or not.\nPremise: Two women are embracing while holding to go packages.\nHypothesis: The sisters are hugging goodbye while holding to go packages after just eating lunch.\nChoose the best answer:\nIf Hypothesis entails the Premise, Answer: 0\nIf Neutral, Answer: 1\nIf Hypothesis does not entail the Premise, Answer: 2\nAnswer only one numerical value. \n\nAnswer: 1', 'label': 

In [15]:
def tokenize_dataset(prompts, labels, tokenizer, max_length=128):
    encodings = tokenizer(
        prompts,
        padding="max_length",          # Pads all sequences to the max length specified
        truncation=True,               # Truncates sequences longer than the max length
        max_length=max_length,         # Sets the maximum sequence length
        return_tensors="pt"           # Returns PyTorch tensors
    )
    
    # Add labels to the encodings
    encodings['labels'] = torch.tensor(labels)  # Ensure labels are added as a tensor
    return encodings

In [16]:
train_prompts = [example['prompt'] for example in dataset_prompt['train']]
train_labels = [example['label'] for example in dataset_prompt['train']]
val_prompts = [example['prompt'] for example in dataset_prompt['val']]
val_labels = [example['label'] for example in dataset_prompt['val']]
test_prompts = [example['prompt'] for example in dataset_prompt['test']]
test_labels = [example['label'] for example in dataset_prompt['test']]

In [17]:
train_encodings = tokenize_dataset(train_prompts, train_labels, tokenizer)
val_encodings = tokenize_dataset(val_prompts, val_labels, tokenizer)
test_encodings = tokenize_dataset(test_prompts, test_labels, tokenizer)
print(train_encodings)
# Display a sample of the tokenized train dataset
print("Sample tokenized input IDs for Train Dataset:\n", train_encodings['input_ids'][0])
print("\nSample attention mask for Train Dataset:\n", train_encodings['attention_mask'][0])
print(train_encodings['labels'][0])

{'input_ids': tensor([[ 5569,   262,  1708,  ..., 50256, 50256, 50256],
        [ 5569,   262,  1708,  ..., 50256, 50256, 50256],
        [ 5569,   262,  1708,  ..., 50256, 50256, 50256],
        ...,
        [ 5569,   262,  1708,  ..., 50256, 50256, 50256],
        [ 5569,   262,  1708,  ..., 50256, 50256, 50256],
        [ 5569,   262,  1708,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 2, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 2, 0,
        1, 0, 2, 2, 0, 0, 0, 2, 1, 2, 0, 1, 0, 2, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1,
        1, 0, 2, 2, 2, 1, 0, 1, 2, 0, 2, 2, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 0, 1,
        1, 2, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0,
        1, 1, 2, 1, 2, 2, 1, 0, 1, 2, 1, 0, 2, 2, 0, 2, 

In [18]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create datasets
train_dataset_custom = CustomDataset(train_encodings)
val_dataset_custom = CustomDataset(val_encodings)
test_dataset_custom = CustomDataset(test_encodings)


In [19]:
import transformers
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_custom,
    eval_dataset=val_dataset_custom,
    peft_config=peft_config,
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
import time

# Measure time taken for fine-tuning
start_time = time.time()

# Fine-tune model
trainer.train()

# Time taken for fine-tuning
time_taken = time.time() - start_time
print(f"Time taken for fine-tuning: {time_taken:.2f} seconds")


# Save final model
trainer.save_model("./phi2_finetuned/final_model")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Wa

Epoch,Training Loss,Validation Loss
1,0.426900,0.610361
2,0.287900,0.601027
3,0.279400,0.599365


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/parall

In [ ]:
# Get model parameter counts
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [ ]:
from peft import PeftModel

f_model = AutoModelForCausalLM.from_pretrained('/kaggle/working/phi2_finetuned/final_model').to("cuda")
# f_model = PeftModel.from_pretrained(model, "/kaggle/working/phi2_finetuned/final_model",torch_dtype=torch.float16,is_trainable=False)


In [ ]:
def generate_prompt(premise, hypothesis):
    return (
        f"Read the following Premise and Hypothesis and determine whether Hypothesis entails the Premise or not.\n"
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        f"Choose the best answer:\n"
        f"If Hypothesis entails the Premise, Answer: 0\n"
        f"If Neutral, Answer: 1\n"
        f"If Hypothesis does not entail the Premise, Answer: 2\n"
        f"Answer only one numerical value. \n\n"
        f"Answer:"
    )

In [ ]:
import re

# Define mapping of labels
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
finetune_outputs = []
# Evaluate on the test set
correct = 0
total = 0

for example in dataset['test']:
    # Create prompt from premise and hypothesis
    prompt = generate_prompt(example['premise'], example['hypothesis'])
    
    # Generate model's response
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = f_model.generate(**inputs, max_new_tokens=15)
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip().lower()
#     print(f"Model response: {response}")

    # Find the last occurrence of "answer: 0", "answer: 1", or "answer: 2" in reverse order
    last_answer = None

    for ans in ["answer: 0", "answer: 1", "answer: 2"]:
        pos = response.rfind(ans)  # Search from the end of the text
        if pos != -1:
            # Keep track of the last found answer if it appears later in the text
            if last_answer is None or pos > response.rfind(last_answer):
                last_answer = ans

    output = None
    if(last_answer == "answer: 0"):
        output = 0
    elif(last_answer == "answer: 1"):
        output = 1
    elif(last_answer == "answer: 2"):
        output = 2
    
#     print(output)
#     print(example['label'])
    if output == example['label']:
        correct += 1
    total += 1
    finetune_outputs.append(output)
# Calculate and print accuracy
accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
dataset['test'][1]

In [ ]:
# Define mapping of labels
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
).to(device)


for i in range(100):
    true_label = dataset['test'][i]['label']
    if(pretrain_outputs[i] != true_label and finetune_outputs[i] == true_label):
        prompt = generate_prompt(dataset['test'][i]['premise'], dataset['test'][i]['hypothesis'])

    # Pretrained model inference
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        output_pretrained = model.generate(**inputs, max_new_tokens=15)
        response_pretrained = tokenizer.decode(output_pretrained[0], skip_special_tokens=True).strip().lower()

        # Fine-tuned model inference
        output_finetuned = f_model.generate(**inputs, max_new_tokens=15)
        response_finetuned = tokenizer.decode(output_finetuned[0], skip_special_tokens=True).strip().lower()
        print(f"True Label: {true_label}")
        print(f"Pretrained:\n {response_pretrained}")
        print(f"Finetuned:\n {response_finetuned}")
        
    if(pretrain_outputs[i] != true_label and finetune_outputs[i] != true_label):
        prompt = generate_prompt(dataset['test'][i]['premise'], dataset['test'][i]['hypothesis'])

    # Pretrained model inference
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        output_pretrained = model.generate(**inputs, max_new_tokens=15)
        response_pretrained = tokenizer.decode(output_pretrained[0], skip_special_tokens=True).strip().lower()

        # Fine-tuned model inference
        output_finetuned = f_model.generate(**inputs, max_new_tokens=15)
        response_finetuned = tokenizer.decode(output_finetuned[0], skip_special_tokens=True).strip().lower()
        print(f"True Label: {true_label}")
        print(f"Pretrained:\n {response_pretrained}")
        print(f"Finetuned:\n {response_finetuned}")
    
